In [1]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel, PeftConfig
from trl import SFTTrainer
import time
from dotenv import load_dotenv
import random
random.seed(42)

load_dotenv()

base_model = "meta-llama/Llama-2-7b-chat-hf"
new_model = "Llama-2-7b-chat-hf-text2sql"
token = os.getenv('token')

/Users/leo/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
peft_params = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model_id = new_model
config = LoraConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, 
                                             return_dict=True, 
                                             #load_in_8bit=True, 
                                             #device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
#peft_model = PeftModel.from_pretrained(model, peft_model_id)
#del model

ValueError: Can't find 'adapter_config.json' at 'Llama-2-7b-chat-hf-text2sql'

In [ ]:
pipe = pipeline(task="text-generation", model=peft_model, tokenizer=tokenizer, max_length=2000)

In [ ]:
prompt = "[INST]<<SYS>>\nBelow are sql tables schemas paired with instruction that describes a task. \n                Using valid SQLite, write a response that appropriately completes the request \n                for the provided tables. <</SYS>>\n\nWhat was round 7's lowest overall?CREATE TABLE table_name_72 (\n    overall INTEGER,\n    round VARCHAR\n)[/INST]"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])